### Basics Imports

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')
import numpy as np
import pandas as pd
import sys

### TensorNetworks imports

In [ ]:
sys.path.append("../")
import tensornetworks as tn
import tn_quantum_circuits as tnqc
import gates as gt
from tensornetworks import contract_indices

### Imports QLM

In [ ]:
import qat.lang.AQASM as qlm
from qat.qpus import PyLinalg
qpu_p = PyLinalg()
from qlm_stuff import proccess_qresults

In [ ]:
def apply_2qubit_gates(qubits, gates):
    """
    Executes product of tensor with a gate
    -o-o-o-o-o-..o-o-
     |   |   |     |
    """
    new_qubits = [0 for i in qubits]
    left = qubits[0]
    for i in range(1, len(qubits)):
        right = qubits[i]
        gate = gates[i-1]
        #new_qubits[i-1], left = phase_change(left, right, gate)
        new_qubits[i-1], left = tnqc.apply_2qubit_gate(left, right, gate)

    new_qubits[-1], new_qubits[0] = tnqc.apply_2qubit_gate(
        left, new_qubits[0], gates[-1])
     #new_qubits[-1], new_qubits[0] = phase_change(left, new_qubits[0], gates[-1])
    return new_qubits

In [ ]:
def get_angles(depth):
    theta = np.pi/4.0
    delta_theta = theta / (depth + 1)
    angles = []
    for i in range(depth):
        angles.append([(2 * i + 1) * delta_theta, (2 * i + 2) * delta_theta])
    return angles     

In [ ]:
get_angles(4)

In [ ]:
def ansatz(nqubits, depth, angles):
    # Intitial State
    zeroket = np.zeros((1, 2, 1))
    zeroket[0][0][0] = 1
    zeroket = zeroket.astype(complex)
    #Initial State
    mps_ = [zeroket] * nqubits
    for depth_ in range(depth):
        # First Layer
        gates = [gt.x_rotation(angles[depth_][0]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
        ent_gates = [gt.controlz() for i in mps_]
        mps_ = apply_2qubit_gates(mps_, ent_gates)
        gates = [gt.z_rotation(angles[depth_][1]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
    return mps_

In [ ]:
def ansatz_qlm(nqubits, depth, angles):
    qprog = qlm.Program()
    qbits = qprog.qalloc(nqubits)
    for d_ in range(0, depth):
        for i in range(nqubits):
            qprog.apply(qlm.RX(angles[d_][0]), qbits[i])
        for i in range(nqubits-1):
            qprog.apply(qlm.Z.ctrl(), qbits[i], qbits[i+1])    
        qprog.apply(qlm.Z.ctrl(), qbits[nqubits-1], qbits[0])
        for i in range(nqubits):
            qprog.apply(qlm.RZ(angles[d_][1]), qbits[i])    
    circ = qprog.to_circ()
    #%qatdisplay circ
    job = circ.to_job()
    state = qpu_p.submit(job)
    pdf = proccess_qresults(state, nqubits)
    pdf.reset_index(drop=True, inplace=True)
    return pdf, circ  

In [ ]:
depth = 3
nqubits = 4 
mps = ansatz(nqubits, depth, get_angles(depth))
pdf_zalo = tnqc.get_state_from_mps(mps)

In [ ]:
pdf, c= ansatz_qlm(nqubits, depth,  get_angles(depth))

In [ ]:
%qatdisplay c --svg

In [ ]:
np.isclose(pdf["Amplitude"], pdf_zalo["Amplitude"]).all()

In [ ]:
[mps_.shape for mps_ in mps]

### Testing  Isometry

In [ ]:
# Identitidad: U.T @ U
tensor = mps[3]
iso = tn.contract_indices(tensor, tensor.conj(), [0, 1], [0, 1])
np.isclose(iso, np.eye(iso.shape[0])).all()

In [ ]:
#Proyector U @U.T
tensor = mps[3]
iso = tn.contract_indices(tensor, tensor.conj(), [2], [2])
projector = iso.reshape(np.product(iso.shape[:2]), np.product(iso.shape[:2]))
np.isclose(projector @ projector, projector).all()

In [ ]:
# Identitidad: U.T @ U
tensor = mps[0]
iso = tn.contract_indices(tensor, tensor.conj(), [1, 2], [1, 2])
np.isclose(iso, np.eye(iso.shape[0])).all()

## Computing density matrices

In [ ]:
[mps_.shape for mps_ in mps]

In [ ]:
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(amp.shape[0], 1)
rho0 = amp @ amp.conj().T

In [ ]:
# Computing Density Matrix
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(tuple([2 for i in range(nqubits)]))
rho1 = tn.reduced_matrix(amp, [0, 1, 2, 3], [])

In [ ]:
np.isclose(rho0, rho1).all()

In [ ]:
amp.shape

In [ ]:
import copy

In [ ]:
def compose_from_right(mps_):
    mps = copy.deepcopy(mps_)
    mps.reverse()
    tensor = mps[0]
    tensor_out = contract_indices(tensor, tensor.conj(), [1, 2], [1, 2])
    for tensor in mps[1:]:
        tensor_out = contract_indices(tensor, tensor_out, [2], [0])
        tensor_out = contract_indices(tensor_out, tensor.conj(), [1, 2], [1, 2])
    return tensor_out
def opa(mps):
    tensor = mps[0]
    tensor_out = contract_indices(tensor, tensor.conj(), [0], [0])
    print(tensor_out.shape, tensor.shape)
    first_leg = 1
    for tensor in mps[1:]:
        print(tensor_out.shape, tensor.shape, first_leg)
        tensor_out = contract_indices(tensor_out, tensor, [first_leg], [0])
        first_leg = first_leg + 1
        print(tensor_out.shape, tensor.shape, tensor_out.ndim-1)
        tensor_out = contract_indices(tensor_out, tensor.conj(), [tensor_out.ndim-1], [0])
    return tensor_out, first_leg        

In [ ]:
traced_out = compose_from_right(mps[2:])

In [ ]:
traced_out.shape

In [ ]:
len(mps)

In [ ]:
len(mps[2:])

In [ ]:
len(mps[0:2])

In [ ]:
non_traced_out, leg = opa(mps[0:2])

In [ ]:
non_traced_out.shape

In [ ]:
leg

In [ ]:
contract_indices(non_traced_out, traced_out, [leg, non_traced_out.ndim-1], [0, 1])

In [ ]:
tn.reduced_matrix(amp, [0], [1, 2, 3])

In [ ]:
tn.reduced_matrix?

In [ ]:
contract_indices(non_traced_out, traced_out, [leg, non_traced_out.ndim-1], [0, 1]) / tn.reduced_matrix(amp, [0], [1, 2, 3])

In [ ]:
traced_out.shape

In [ ]:
a = compose_from_right(mps)

In [ ]:
rho_t = tnqc.compose_mps(mps)
rho_t = contract_indices(rho_t, rho_t.conj(), [], [])
rho_t= rho_t.reshape(np.prod(rho_t.shape[:rho_t.ndim//2]), -1)

In [ ]:
np.isclose(rho0, rho_t).all()

In [ ]:
rho_t.shape

In [ ]:
[mps_.shape for mps_ in mps]

In [ ]:
a

In [ ]:
rho0

In [ ]:
contract_indices(amp, amp.conj(), )

In [ ]:
non_contracted = 1
tensor = mps[:non_contracted]
b = tnqc.compose_mps(mps[non_contracted:], False)

In [ ]:
b.shape

In [ ]:
[t.shape for t in tensor]

In [ ]:
contract_indices(b)

In [ ]:
b.shape

In [ ]:
np.isclose(a.reshape(2**4, 2**4), rho0).all()

In [ ]:
forrho = [mps[0]]a
b = tnqc.compose_mps(forrho)
a = contract_indices(np.conj(mps[0]),mps[0],  [0, 2], [0, 2])

In [ ]:
a

In [ ]:
mps[0].shape

In [ ]:
tn.reduced_matrix(amp, [0], [1, 2, 3])

In [ ]:
len(forrho)

In [ ]:
contract_indices?

In [ ]:
pdf_zalo["Amplitude"]

In [ ]:
a = tn.compose_mps(mps)

In [ ]:
b = a.reshape(np.prod(a.shape))

In [ ]:
b

In [ ]:
a = a.reshape(a.shape[0], -1, a.shape[-1])

In [ ]:
a.shape

In [ ]:
a.shape

In [ ]:
opa = contract_indices(a.conj(), a,  [0, 2], [0, 2])

In [ ]:
opa

In [ ]:
rho1

In [ ]:
amp.shape

In [ ]:
a = tn.compose_mps(mps)

In [ ]:
a.ndim

In [ ]:
mps

In [ ]:
non_contracted = 1
tensor = mps[:non_contracted]
b = compose_from_right(mps[non_contracted:])

In [ ]:
b.shape

In [ ]:
c = opa(tensor)

In [ ]:
contract_indices(c, b, [1, 3], [0, 1])

In [ ]:
# Computing Density Matrix
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(tuple([2 for i in range(nqubits)]))
rho1 = tn.reduced_matrix(amp, [0], [1, 2, 3])

In [ ]:
rho1

In [ ]:
c.shape

In [ ]:
def density_matrix(mps):
    tensor = mps[0]
    tensor_out = contract_indices(tensor, tensor.conj(), [0], [0])
    for tensor in mps[1:-1]:
        print(tensor_out.shape)
        print(tensor.shape)
        print(tensor_out.ndim)
        tensor_out = contract_indices(tensor, tensor_out, [0], [1])
        tensor_out = contract_indices(tensor_out, tensor.conj(), [tensor_out.ndim-1], [0])
    tensor = mps[-1]
    tensor_out = contract_indices(tensor, tensor_out, [0], [1])
    tensor_out = contract_indices(tensor_out, tensor.conj(), [tensor_out.ndim-1, 1], [0, 2])
    return tensor_out
        
        
    

In [ ]:
opa = density_matrix(mps)

In [ ]:
rho_mps = opa.reshape(2**(opa.ndim //2), 2**(opa.ndim //2))

In [ ]:
rho_mps

In [ ]:
rho0

In [ ]:
np.isclose(rho0, rho_mps)

In [ ]:
opa.ndim /2

In [ ]:
opa.size